In [1]:

import os
import cv2
import json
import numpy as np
import albumentations as alb
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [3]:
# Load images and annotations
images = ["C:\\Users\\chsur\\OneDrive\\Desktop\\suri-all\\data2\\images\\*.jpg"]
annotations = ["C:\\Users\\chsur\\cv\\data\\labels\\*.json"]

for folder in ['train', 'test', 'val']:
    image_folder = os.path.join('C:\\Users\\chsur\\OneDrive\\Desktop\\suri-all\\data2', folder, 'images')
    label_folder = os.path.join('C:\\Users\\chsur\\OneDrive\\Desktop\\suri-all\\data2', folder, 'labels')

    for file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, file)
        images.append(cv2.imread(image_path))

        label_path = os.path.join(label_folder, file.split('.')[0] + '.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                annotations.append(json.load(f))

In [4]:
print(image_folder)

C:\Users\chsur\OneDrive\Desktop\suri-all\data2\val\images


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

num_samples = len(images)

# Split dataset into training, testing, and validation sets
test_size = 2  # Adjusting test_size to avoid empty train set
X_train, X_testval, y_train, y_testval = train_test_split(images, annotations, test_size=test_size, random_state=4)
X_test, X_val, y_test, y_val = train_test_split(X_testval, y_testval, test_size=0.5, random_state=42)


ValueError: test_size=2 should be either positive and smaller than the number of samples 1 or a float in the (0, 1) range

In [6]:
# Define augmentation pipeline
augmentor = alb.Compose([
    alb.RandomCrop(width=450, height=450),
    alb.HorizontalFlip(p=0.5),
    alb.RandomBrightnessContrast(p=0.2),
    alb.RandomGamma(p=0.2),
    alb.RGBShift(p=0.2),
    alb.VerticalFlip(p=0.5)],
    bbox_params=alb.BboxParams(format='albumentations', label_fields=['class_labels']))

In [ ]:
# Augment and save images and annotations
for partition, X, y in [('train', X_train, y_train), ('test', X_test, y_test), ('val', X_val, y_val)]:
    for idx, (image, annotation) in enumerate(zip(X, y)):
        try:
            coords = [0, 0, 0.00001, 0.00001]
            if 'shapes' in annotation and len(annotation['shapes']) > 0:
                coords[0] = annotation['shapes'][0]['points'][0][0]
                coords[1] = annotation['shapes'][0]['points'][0][1]
                coords[2] = annotation['shapes'][0]['points'][1][0]
                coords[3] = annotation['shapes'][0]['points'][1][1]
                coords = list(np.divide(coords, [640, 480, 640, 480]))

            for x in range(60):
                augmented = augmentor(image=image, bboxes=[coords], class_labels=['face'])

                cv2.imwrite(os.path.join("C:\\Users\\chsur\\cv\\aug_data", partition, 'images', f'{idx}.{x}.jpg'), augmented['image'])

                annotation_dict = {
                    'image': f'{idx}.{x}.jpg',
                    'bbox': augmented['bboxes'][0] if augmented['bboxes'] else [0, 0, 0, 0],
                    'class': 1 if augmented['bboxes'] else 0
                }

                with open(os.path.join('aug_data', partition, 'labels', f'{idx}.{x}.json'), 'w') as f:
                    json.dump(annotation_dict, f)
        except Exception as e:
            print(e)


In [ ]:
# Load augmented data and create TensorFlow datasets
def load_image_and_label(image_path, label_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, (120, 120))
    image = image / 255.0

    with open(label_path.numpy(), 'r') as f:
        label = json.load(f)

    return image, [label['class']], label['bbox']

train_images = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/train/images/*.jpg')
train_labels = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/train/labels/*.json')
train_dataset = tf.data.Dataset.zip((train_images, train_labels)).map(
    lambda x, y: tf.py_function(load_image_and_label, [x, y], [tf.float32, tf.uint8, tf.float16]))

test_images = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/test/images/*.jpg')
test_labels = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/test/labels/*.json')
test_dataset = tf.data.Dataset.zip((test_images, test_labels)).map(
    lambda x, y: tf.py_function(load_image_and_label, [x, y], [tf.float32, tf.uint8, tf.float16]))

val_images = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/val/images/*.jpg')
val_labels = tf.data.Dataset.list_files('C:\\Users\\chsur\\cv\\aug_data/val/labels/*.json')
val_dataset = tf.data.Dataset.zip((val_images, val_labels)).map(
    lambda x, y: tf.py_function(load_image_and_label, [x, y], [tf.float32, tf.uint8, tf.float16]))


In [ ]:
# Build the model
def build_model():
    input_layer = Input(shape=(120, 120, 3))
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)

    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)

    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

facetracker = build_model()
facetracker.compile(optimizer='adam', loss=['binary_crossentropy', localization_loss], metrics=['accuracy']) # type: ignore

# Train the model
facetracker.fit(train_dataset.batch(8), epochs=10, validation_data=val_dataset.batch(8))

# Evaluate the model
facetracker.evaluate(test_dataset.batch(8))
